<a href="https://colab.research.google.com/github/porameht/SFT-customer-support-26.9k/blob/main/SFT_optg_7b_customer_support_26_9k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!nvidia-smi

Sun May  5 01:48:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, gradio, warnings
from datasets import load_dataset,Dataset
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
from huggingface_hub import notebook_login
import pandas as pd


In [4]:
notebook_login()

In [21]:
model_name = "openthaigpt/openthai-llama-pretrained-7B"

dataset_name = "Porameht/customer-support-th-26.9k"

new_model = "openthaigpt-7b-customer-support-th"

In [6]:
# Load dataset
dataset=load_dataset(dataset_name,split='train')

df_train = dataset.to_pandas()
train_data, eval_data = train_test_split(df_train, test_size=0.2, random_state=42)

#perform preprocessing
def data_preprocessing(text):

  full_prompt =f"""<s>[INST]{text['instruction']}
              [/INST] {text['response']}
              </s>"""

  return {"text": full_prompt}


# Apply processing function to each row using apply_rows
train_dataset = Dataset.from_pandas(train_data).map(data_preprocessing)
eval_dataset = Dataset.from_pandas(eval_data).map(data_preprocessing)


# Print the first few rows to verify the preprocessing
print(train_dataset["text"][:5])
print(eval_dataset["text"][:5])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

Map:   0%|          | 0/21497 [00:00<?, ? examples/s]

Map:   0%|          | 0/5375 [00:00<?, ? examples/s]

['<s>[INST]ต้องการความช่วยเหลือในการดูค่าธรรมเนียมการเลิกจ้างก่อนกำหนด\n              [/INST] เพื่อช่วยให้คุณเข้าใจค่าธรรมเนียมการยกเลิกก่อนกำหนด โปรดระบุรายละเอียดบัญชีของคุณเพื่อให้เราสามารถเข้าถึงข้อกำหนดและเงื่อนไขเฉพาะที่เกี่ยวข้องกับแผนของคุณ เมื่อเรามีข้อมูลนี้แล้ว เราจะสามารถให้คำอธิบายโดยละเอียดเกี่ยวกับค่าธรรมเนียมหรือบทลงโทษที่อาจเกิดขึ้นได้\n              </s>', '<s>[INST]คุณช่วยฉันอัปเดตรายละเอียดที่รวมอยู่ในบัญชีของฉันได้ไหม\n              [/INST] ฉันอยู่บนนั้น! เราพร้อมช่วยเหลือคุณในการอัปเดตรายละเอียดที่รวมอยู่ในบัญชีของคุณ ความมุ่งมั่นของคุณในการรักษาข้อมูลของคุณให้ถูกต้องและเป็นส่วนตัวนั้นน่ายกย่อง เพื่อความปลอดภัยสูงสุดของบัญชีของคุณ เราขอให้คุณอย่าเปิดเผยตัวระบุส่วนบุคคลใด ๆ เช่น หมายเลขบัญชีเต็มหรือรหัสผ่านของคุณที่นี่ หากต้องการอัปเดตรายละเอียดบัญชีของคุณ ทางที่ดีที่สุดคือเข้าสู่ระบบ {{Online Company Portal Info}} อย่างปลอดภัย หรือติดต่อทีมสนับสนุนลูกค้าของเราเพื่อขอความช่วยเหลือเฉพาะบุคคล คุณต้องการอัปเดตรายละเอียดเฉพาะด้านใดบ้าง เราจะแนะนำคุณตลอดกระบวนการทีละขั้

In [7]:
# Load base model and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(True, True)

In [8]:
#monitering login
wandb.login(key="a3805f62765554898d49d4154162b9a670ad6df6")
run = wandb.init(project='openthai-gpt', job_type="training", anonymous="allow")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fr4nk (service-engineering). Use `wandb login --relogin` to force relogin


In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(56541, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [10]:
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [ ]:
# adafactor=False,
# adam_beta1=0.9,
# adam_beta2=0.999,
# adam_epsilon=1e-08,
# auto_find_batch_size=False,
# bf16=True,
# bf16_full_eval=False,
# data_seed=None,
# dataloader_drop_last=False,
# dataloader_num_workers=0,
# dataloader_persistent_workers=False,
# dataloader_pin_memory=True,
# dataloader_prefetch_factor=None,
# dataset_kwargs=None,
# ddp_backend=None,
# ddp_broadcast_buffers=None,
# ddp_bucket_cap_mb=None,
# ddp_find_unused_parameters=None,
# ddp_timeout=1800,
# debug=[],
# deepspeed=None,
# disable_tqdm=False,
# dispatch_batches=None,
# do_eval=False,
# do_predict=False,
# do_train=False,
# eval_accumulation_steps=None,
# eval_delay=0,
# eval_do_concat_batches=True,
# eval_steps=None,
# evaluation_strategy=no,
# fp16=False,
# fp16_backend=auto,
# fp16_full_eval=False,
# fp16_opt_level=O1,
# fsdp=[],
# fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
# fsdp_min_num_params=0,
# fsdp_transformer_layer_cls_to_wrap=None,
# full_determinism=False,
# gradient_accumulation_steps=32,
# gradient_checkpointing=True,
# gradient_checkpointing_kwargs={'use_reentrant': False},
# greater_is_better=None,
# group_by_length=False,
# half_precision_backend=auto,
# hub_always_push=False,
# hub_model_id=None,
# hub_private_repo=False,
# hub_strategy=every_save,
# hub_token=<HUB_TOKEN>,
# ignore_data_skip=False,
# include_inputs_for_metrics=False,
# include_num_input_tokens_seen=False,
# include_tokens_per_second=False,
# jit_mode_eval=False,
# label_names=None,
# label_smoothing_factor=0.0,
# learning_rate=0.0002,
# length_column_name=length,
# load_best_model_at_end=False,
# local_rank=0,
# log_level=info,
# log_level_replica=warning,
# log_on_each_node=True,
# logging_dir=data/typhoon-7b-sft-qlora/runs/Apr27_15-50-20_eef658a497e1,
# logging_first_step=True,
# logging_nan_inf_filter=True,
# logging_steps=50,
# logging_strategy=steps,
# lr_scheduler_kwargs={},
# lr_scheduler_type=cosine,
# max_grad_norm=1.0,
# max_seq_length=2048,
# max_steps=-1,
# metric_for_best_model=None,
# mp_parameters=,
# neftune_noise_alpha=None,
# no_cuda=False,
# num_train_epochs=1,
# optim=adamw_torch,
# optim_args=None,
# optim_target_modules=None,
# output_dir=data/typhoon-7b-sft-qlora,
# overwrite_output_dir=True,
# past_index=-1,
# per_device_eval_batch_size=2,
# per_device_train_batch_size=2,
# prediction_loss_only=False,
# push_to_hub=False,
# push_to_hub_model_id=None,
# push_to_hub_organization=None,
# push_to_hub_token=<PUSH_TO_HUB_TOKEN>,
# ray_scope=last,
# remove_unused_columns=True,
# report_to=['wandb'],
# resume_from_checkpoint=None,
# run_name=data/typhoon-7b-sft-qlora,
# save_on_each_node=False,
# save_only_model=False,
# save_safetensors=True,
# save_steps=10,
# save_strategy=steps,
# save_total_limit=1,
# seed=42,
# skip_memory_metrics=True,
# split_batches=None,
# tf32=None,
# torch_compile=False,
# torch_compile_backend=None,
# torch_compile_mode=None,
# torchdynamo=None,
# tpu_metrics_debug=False,
# tpu_num_cores=None,
# use_cpu=False,
# use_ipex=False,
# use_legacy_prediction_loop=False,
# use_mps_device=False,
# warmup_ratio=0.1,
# warmup_steps=0,
# weight_decay=0.0,




training_arguments = TrainingArguments(
    output_dir= "./output",
    num_train_epochs= 1,
    # per_device_eval_batch_size=2,
    # per_device_train_batch_size=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    optim="adamw_torch",
    save_steps= 10,
    logging_steps= 25,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 1.0,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "cosine",
    report_to="wandb"
)

#finetuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length= None,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/21497 [00:00<?, ? examples/s]

Map:   0%|          | 0/5375 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,3.241900
50,3.720600
75,2.758900
100,2.406100
125,2.037100
150,1.795800
175,1.686200
200,1.531400
225,1.523100
250,1.402000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=Fals

In [22]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
# model.config.use_cache = True
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(56541, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [23]:
trainer.model.push_to_hub(new_model)
trainer.tokenizer.push_to_hub(new_model)


README.md:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Porameht/openthaigpt-7b-customer-support-th/commit/87f2bb2802e469f9f637b69c5ef5ea977569fe0f', commit_message='Upload tokenizer', commit_description='', oid='87f2bb2802e469f9f637b69c5ef5ea977569fe0f', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

In [24]:
# Load base model with adopter
base_model_reload = AutoModelForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)

model = PeftModel.from_pretrained(base_model_reload, new_model)

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [25]:
model = model.merge_and_unload()

model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

model-00003-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Porameht/openthaigpt-7b-customer-support-th/commit/5ecb6b9620eb0cd8647eaa9e8049f749231046b9', commit_message='Upload tokenizer', commit_description='', oid='5ecb6b9620eb0cd8647eaa9e8049f749231046b9', pr_url=None, pr_revision=None, pr_num=None)